### Получение сборок.
1. Получение двух фаста файлов, списка всех UCE, файл со всеми ридами. 
   
   (cat IonXpress_\d\d\d_rawlib.basecaller_unadapt_trimmed0_0.cor.fasta > All_spicies.fasta)

    -> All_spicies.fasta
    -> All_UCE.fasta
    
2. Сборка de novo через Trinity. 

   (Trinity --seqType fa --single All_spicies.fasta) 
   
   -> All_spicies_Trinity.fasta
    
3. Поиск включений UCE в сборке через бласт (через биопитон?). 
   
   (makeblastdb  -in 'All_spicies_Trinity.fasta' -dbtype nucl -out blast_index, 
   
   blastn -query All_UCE.fasta -db blast_index -outfmt 6 -out UCE_in_Trinity.outfmt6)
   
   -> UCE_in_Trinity.outfmt6
   
4. Из UCE_in_Trinity.outfmt6 и All_spicies_Trinity.fasta составление фаста файлов с контигами, содержащими UCE, для каждого UCE.

   (with open UCE_in_Trinity.outfmt6:
     по имени UCE в первой строчке, создаем фаста файл, куда записываем из файла All_spicies_Trinity.fasta контиги, найденные в UCE_in_Trinity.outfmt6 по шаблону второй строки)
   
   -> UCE№_by_Trinity.fasta (по числу UCE)
   
5. Полученные файлы помещаю в CD-HIT-EST 
   
   (cd-hit-est -i UCE№_by_Trinity.fasta -c 0.8 -l 200 -r 1 -d 30)
   
6. Делаю фаста файлы с кластерами. 
   
   (в файле )
   
   -> UCE№_by_Trinity_Claster№.fasta (по числу кластеров для каждого UCE * число UCE)
   
7. Загружаю файлы с кластерами в сар3. 
   
   (cap3 UCE№_by_Trinity_Claster№.fasta  -p 85 -f 1500 -e 500 -z 1 -o 16 -s 251)
   
   -> UCE№_by_Trinity_cap3_Cosensus№.fasta
 
8. Для каждого консенсуса делаю сборку на него каждого из видов в BWA. 
   
   (bwa index UCE№_by_Trinity_cap3_Cosensus№.fasta
   
   bwa mem UCE№_by_Trinity_cap3_Cosensus№.fasta SP№.fasta > UCE№_SP№_Сonsensus№.sam
   samtoola view -F 4 UCE№_SP№_Сonsensus№.sam > UCE№_SP№_Сonsensus№_mapping.sam)
   
   -> UCE№_SP№_Сonsensus№_mapping.sam

Два основных модуля. 
1. Получение сборок
2. Чистка сборок

### Чистка сборок (прописано для одной сборки, повторяем для всех)
Наверно через биопитон или через pysam

1. Делаем список, со значениями покрытия для каждого нуклеотида. (samtools depth) (для частоты встерчаемости замен)

2. Каждый нуклеотид сравниваю с референсом. Составляю список с кортежами замен [(53, 'A'), (100, 'T')...]

Готово!

3. Смотрю, насколько часто встерчается подобная замена в целом. Нужно: 
    А. Посмотреть сколько всего замен в каждом риде. Сделать список с номером и типом замены для каждого рида. 
    Б. Для каждого нуклеотида в референсе посмотерть на тип его замены и соотношение типов замен.
    
4. Если замена харктерна для 10%-40% покрытия, возможно, риды с такое заменой принадлежат другой хромосоме. (их и ищем, чтобы удалить)
   
   Если замена характерна для 40%-60% покрытия, это скорее всего гетерозиготность. (или разница A и B хромосом)
   
   Если замена характерна для 100% покрытия, это отличие от референса, характерное для вида. (Этот тип замен можно удалить из словарей замен)
   
   Если замена характерна для <10% покрытия, это возможно ошибки в гомополимерах. (проверяем соседние нуклеотиды, если справа или слева идут такие же нуклеотиды, то скорее всего    это просто ошибка)
   
   Если замена характерна для 60% < 90% покрытия, возможно это тоже межвидовые различия
   
   

5. Смотрим на замены характерные менее чем для 10 % ридов. Если в словарях, где есть эти замены, есть еще замены менее 10%, то рид выкидываем.


Что делать если рида в сборке всего два?
Call_bases
Очистить sam файл, тогда колонок будет одинакого. Убрать первую строку.

In [1]:
import pysam
import Bio
import pandas as pd
from Bio import SeqIO

In [3]:
# Загружаем файл, берем из него колонки с ридом, и позицией начала рида Флаги!
data = pd.read_csv('UCE4_sp3_trinity_cap3_contig1_cart.sam', sep = '\t', error_bad_lines=False)
data = pd.concat([pd.DataFrame([data.columns.values], columns=data.columns), data], ignore_index=True) # сдвигаем данные (название столбцов это часть данных)


start = data.iloc[:, 3] # столбец из sam, где начало рида
seq = data.iloc[:, 9] # столбец из sam c самим ридом
cigar = data.iloc[:, 5] # столбец из sam с строкой cigar (ищем гапы)
flag = data.iloc[:, 1] # столбец из sam с строкой cigar (ищем гапы)

data =  start.to_frame().join(seq.to_frame()).join(cigar.to_frame()).join(flag.to_frame())
data.columns = ['Начало рида', 'Рид', 'CIGAR','Flag']
data['Начало рида'].iloc[0] = int(data['Начало рида'].iloc[0])

data

/tmp/ipykernel_113954/3003668664.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('UCE4_sp3_trinity_cap3_contig1_cart.sam', sep = '\t', error_bad_lines=False)
b'Skipping line 107: expected 15 fields, saw 16\nSkipping line 108: expected 15 fields, saw 16\n'


,Начало рида,Рид,CIGAR,Flag
0,138,AATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGA...,72M,16
1,113,TAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGT...,143M,0
2,77,GTAGGAGGGCTTTATGGAGCAGAAAAACGACAAAGCTAGAAAAATT...,82M,0
3,113,TAGAAAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGT...,162M6I35M,0
4,35,GCTGTGTCACGTGGTTAAGTAACTTTACAGGGTCGCTTGCTAGTAG...,171M,0
...,...,...,...,...
1211,282,ATCACCACCAGGAGTTGTATCATCCGCGAGTGACCTACCAGGAGCG...,52M,16
1212,106,ACAAAGGTAGAAAAAATATTTTTCACTCCAAAAATTAATGATCATG...,213M,0
1213,51,TTAAAGTAACTTTACAGGGTCTCTTGCTAGTAGGAGGTCTTTATGG...,3S151M,16
1214,134,AAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGTATTT...,17S126M14S,16


In [4]:
# загрузим соответствующий референс
ref_fasta= SeqIO.parse(open('UCE4_All_Trinity1.fasta'),'fasta')
for i in ref_fasta:
    bio_ref = i.seq
ref =[]
for i in bio_ref:
    ref.append(i)
#ref

In [5]:
def working_cigar(cigar):
    '''For input the cigar string, for output the cigar dictionary'''
    cigar = list(cigar.strip()) #сделали список
    # для начала нужно сигар разбить на блоки и переделать цифры в инт. По идее олжно получиться int(61) str(S) int(37) str(M)...
    # Каждый символ пытаюсь перевести в инт, обработка исключений на буквы
    for i in range(len(cigar)): #пробуем каждый символ списка сделать интом
        try:
            cigar[i] = int(cigar[i])
        except ValueError:
            cigar[i] = cigar[i]
            
    # Потом объединяю цифры в одно число.
    cigar_new = []
    for i in range(len(cigar)): # если символ - строка - записываем его отдельно
        if type(cigar[i]) == str:
            cigar_new.append(cigar[i])                                    
        try:
            if type(cigar[i]) == type(cigar[i+1]) == type(cigar[i+2]) : # если три символа подряд это инт, объединяем их
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])+str(cigar[i+2])))
            if type(cigar[i]) == type(cigar[i+1]) and type(cigar[i]) != type(cigar[i+2]) and type(cigar[i-1]) ==str:  # если два символа подряд одного типа (такое может быть только с инт), объединяем их  
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])))
            if type(cigar[i+1]) == str and type(cigar[i-1]) == str: # если один инт - тоже записываем его
                cigar_new.append(cigar[i])
        except IndexError: # это нужно на случай окончания строки, list index out of range
            continue
    #print(cigar_new) # проверка правильности составления списка из cigar      
    # удобнее сделать словарь
    numbers = cigar_new[::2] # список с числом нуклеотидов
    clipping_type = [] # список с типом совпадений
    for i in range(1, len(cigar_new), 2): 
        clipping_type.append(cigar_new[i]);
    #print(numbers, clipping_type) # проверка правильности списков
    cigar_dict = [] # делаем словарь
    for i in range(len(numbers)):
        cigar_dict.append((numbers[i],clipping_type[i]))
    
    return cigar_dict
#working_cigar(cigar)

In [6]:
def seq_after_cigar(cigar, seq):
    nucl_number = 0 # сразу столбец из data
    print(seq)
    for e in cigar:
        #print(key, value) # првоерка работы функции working_cogar
        if e[1] == 'S':
            seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:]
        if e[1] == 'D':
            seq = seq[:nucl_number] + ' ' * e[0] + seq[nucl_number:]
        if e[1] == 'H':
            continue # это пока, может быть hard clipping не вырезается
        if e[1] == 'I':
            seq = seq[:nucl_number] + seq[nucl_number + e[0]:]
        nucl_number += e[0]
    #print(key, nucl_number)
    seq = seq.replace('N', '') 
    
    return seq
#print(seq_after_cigar(working_cigar(cigar), seq))

In [7]:
# попробуем теперь анализировать так все риды из таблицы.
new_data = []
for i in range(len(data)):
    new_data.append((data['Начало рида'].iloc[i] - 1) * ' ' + seq_after_cigar(working_cigar(data['CIGAR'].iloc[i]),data['Рид'].iloc[i]))

AATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGATCCGAAATTTCCTCCTTGCGAGGAAA
TAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAGTATTACAG
GTAGGAGGGCTTTATGGAGCAGAAAAACGACAAAGCTAGAAAAATTATTTTCCACTCCAGAAATTAATGATCATGAGCTCGT
TAGAAAAATAAATTTCCTCTGCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAATATTACAGCCGCGCAAGGGACCCTGGCTCTGGTTACCAGCATCACCATCAGGAGTTATACCCTCCGCG
GCTGTGTCACGTGGTTAAGTAACTTTACAGGGTCGCTTGCTAGTAGGAGGGTTTTATGGACCAGAAAAACGACAAAGGTAGAAAAATTATTTTTCACTCCAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTAGCTACATCGATCCGAAATTTCCTCCTTGCGAG
GTTCCTTATCCGGGAACTACCTCGCCAGCTGCCATTGGACTGCTGTGTCACGTGGTTAAGTAACTTTACAGGGTCGCTTGCTAGTAGGAGGGTTTTATGGACCAGAAAAACGACAAAGGTAGAAAAATTATTTTTCACTCCAGAAATTAATGATCATGAG
CAGAAATTAATGATCATGAGCTCGTATTTGATGGACTCTACCTACATTGATCCGAAATTTCCTCCATGCGAGGAATATTCGCAAAACAGCTACATCCCCGAGCACAGCCCCGAATATTACAGCCGCGCAAGGGACCCTGGCTCTGGTTACCAGCATCACCATC

In [49]:
ref_cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
for i in range(len(ref) + 1):
    cover_dict = {} # для каждого нуклеотида
    cover_dict['A, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['C, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['G, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['T, нуклеотид № {0:.0f}'.format(i+1)] = 0
    ref_cover_list.append(cover_dict)

for read in new_data:
    for nucl_number, nucl in enumerate(read):
            #print(nucl, nucl_number)
            if nucl == 'A':
                ref_cover_list[nucl_number]['A, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
            if nucl == 'T':
                ref_cover_list[nucl_number]['T, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
            if nucl == 'G':
                ref_cover_list[nucl_number]['G, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
            if nucl == 'C':
                ref_cover_list[nucl_number]['C, нуклеотид № {0:.0f}'.format(nucl_number + 1)]  += 1
ref_cover_list

# посчитаем покрытие для каждого нуклеотида
cover_list = [] # большой словарь для всех нуклеотидов, по всей длине референса
for i in range(len(ref) + 1):
    cover_dict = {} # для каждого нуклеотида
    cover_dict['A, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['C, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['G, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_dict['T, нуклеотид № {0:.0f}'.format(i+1)] = 0
    cover_list.append(cover_dict)


for nucl_number, nucl in enumerate(ref_cover_list):
    cover = 0
    for key, value in nucl.items():
        cover += value
    for key, value in nucl.items():    
        if key == 'A, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
        if key == 'T, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
        if key == 'G, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
        if key == 'C, нуклеотид № {:.0f}'.format(nucl_number + 1):
            if cover == 0:
                cover_list[nucl_number][key] =  0
            else:
                cover_list[nucl_number][key] =  value * 100 / cover
cover_list     

[{'A, нуклеотид № 1': 100.0,
  'C, нуклеотид № 1': 0.0,
  'G, нуклеотид № 1': 0.0,
  'T, нуклеотид № 1': 0.0},
 {'A, нуклеотид № 2': 0.0,
  'C, нуклеотид № 2': 0.0,
  'G, нуклеотид № 2': 0.0,
  'T, нуклеотид № 2': 100.0},
 {'A, нуклеотид № 3': 0.0,
  'C, нуклеотид № 3': 100.0,
  'G, нуклеотид № 3': 0.0,
  'T, нуклеотид № 3': 0.0},
 {'A, нуклеотид № 4': 0.0,
  'C, нуклеотид № 4': 100.0,
  'G, нуклеотид № 4': 0.0,
  'T, нуклеотид № 4': 0.0},
 {'A, нуклеотид № 5': 0.0,
  'C, нуклеотид № 5': 0.0,
  'G, нуклеотид № 5': 100.0,
  'T, нуклеотид № 5': 0.0},
 {'A, нуклеотид № 6': 0.0,
  'C, нуклеотид № 6': 0.0,
  'G, нуклеотид № 6': 100.0,
  'T, нуклеотид № 6': 0.0},
 {'A, нуклеотид № 7': 0.0,
  'C, нуклеотид № 7': 0.0,
  'G, нуклеотид № 7': 100.0,
  'T, нуклеотид № 7': 0.0},
 {'A, нуклеотид № 8': 100.0,
  'C, нуклеотид № 8': 0.0,
  'G, нуклеотид № 8': 0.0,
  'T, нуклеотид № 8': 0.0},
 {'A, нуклеотид № 9': 100.0,
  'C, нуклеотид № 9': 0.0,
  'G, нуклеотид № 9': 0.0,
  'T, нуклеотид № 9': 0.0},
 

In [53]:
# Сравниваю каждый рид с референсом, выписываю замены в стиле:  [(53, 'A'), (100, 'T')...]
changing = []
for nucl_number, nucl in enumerate(ref): # для всех нуклеотидов референса
    for seq_number, seq in enumerate(new_data): # для каждого рида сборки
        #print(seq[nucl_number])
        try:
            if seq[nucl_number] != ' ': # если нуклеотид рида существует в этой позиции
                if nucl != seq[nucl_number]: # если нуклеотид референса не равен нуклеотиду рида
                    changing.append(['Рид№ {0:.0f}'.format(seq_number), 
                                     '{0:s}, нуклеотид № {1:.0f}'.format(seq[nucl_number], nucl_number +1)])                 
        except IndexError:
            continue
#for i in range(len(changing)):
#    print(changing[i])
# из этих данных нужно полуить информацию о заменах в каждом риде, сначала выбираю отсюда элементы для рида 9. Потом смотрю при каких номерах нуклеотидов какой тип замены.

In [40]:
# Отсюда можем получить отдельный набор для замен в каждом риде, и для замен по каждой позиции
# Посмотрим, в каких ридах встречаются замены и сколько в каждом риде замен.
read_with_change = []
for e in changing:
    read_with_change.append(e[0])

change = []
for read in read_with_change:
    change.append((read, read_with_change.count(read)))
read_with_change = set(change) # посмотрели в каких ридах есть замены.
#read_with_change

In [41]:
nucleotide_with_change = []
for e in changing:
    nucleotide_with_change.append(e[-1])

change = []
for read in nucleotide_with_change:
    change.append(('В {0:s}'.format(read), nucleotide_with_change.count(read)))

nucleotide_with_change = list(set(change))
nucleotide_with_change = sorted(nucleotide_with_change, key=lambda a: a[0][-3:])
#nucleotide_with_change

## Задача: убрать из сборки риды, которые не принадлежат этой хромосоме.
Как это понять?
1. Эти риды имеют большое число замен (>10). (Проверяем read with change)
2. Эти замены характерны не более чем для 20% ридов. (nculeotide with change)

In [63]:
# Смотрим на файл read with change, берем риды где больше 10 замен
# Смотрим на покрытия для каждого нуклеотида, если для 80% нуклеотидов с заменами покртыие для такой замены меньше 20%, удаляем рид из сборки. 
# хорошо бы сделать визуализацию

candidate_for_remove = []
for read_tuple in read_with_change:
    if read_tuple[1] >= 10: # Вариабельно, не обязательно 10
        candidate_for_remove.append(read_tuple)

# смотрим на то, какие именно замены в этих ридах с файле changing. Цикл по кандидатам, цикл по changing
# Для каждого совпадения changing[i][0] == candidate_for_remove[i]:
# смотрим на changing[1], сопоставляем номеру нуклеотида и типу замены из cover_list
# если значение из ковер лист меньше 20, фиксируем что для этого рида, +1 плохая замена
# потом смотрим на счетчик для каждого рида, сколько из замен рида имеет 8/10 плохих замен, их удаляем.
for read_number, read in enumerate(candidate_for_remove):
    for i, e in enumerate(changing):
        if read[0] == e[0]: # работает!
            for cover in cover_list:
                for key, value in cover.items():
                    if key == e[1]: # работает
                        if value < 20:
                            print('Число замен в риде:', read[1],'\n', 'Информация о риде и замене:',e, '\n','Процент такой замены в целом:',value)
                            

Число замен в риде: 16 
 Информация о риде и замене: ['Рид№ 29', 'A, нуклеотид № 88'] 
 Процент такой замены в целом: 16.727941176470587
Число замен в риде: 16 
 Информация о риде и замене: ['Рид№ 29', 'A, нуклеотид № 122'] 
 Процент такой замены в целом: 16.622691292875988
Число замен в риде: 16 
 Информация о риде и замене: ['Рид№ 29', 'A, нуклеотид № 124'] 
 Процент такой замены в целом: 16.384915474642394
Число замен в риде: 16 
 Информация о риде и замене: ['Рид№ 29', 'T, нуклеотид № 130'] 
 Процент такой замены в целом: 16.237113402061855
Число замен в риде: 16 
 Информация о риде и замене: ['Рид№ 29', 'G, нуклеотид № 133'] 
 Процент такой замены в целом: 18.53233830845771
Число замен в риде: 11 
 Информация о риде и замене: ['Рид№ 660', 'A, нуклеотид № 88'] 
 Процент такой замены в целом: 16.727941176470587
Число замен в риде: 11 
 Информация о риде и замене: ['Рид№ 660', 'A, нуклеотид № 122'] 
 Процент такой замены в целом: 16.622691292875988
Число замен в риде: 11 
 Информация